In [8]:
import pandas as pd
import dataframe_image as dfi

## Model infos for cancer type
loaded from Model Annotation → list of all annotated models

Needed for filtering `all data` for lung cancer

In [9]:
file = "../import_data/CMP/model_list_20240110.csv"

# load data
df_model_info = pd.read_csv(file, delimiter=',', usecols=['model_id', 'tissue', 'cancer_type', 'tissue_status', 'cancer_type_detail'])


# filter for lung cancer
lung_cancer = ['Small Cell Lung Carcinoma', 'Non-Small Cell Lung Carcinoma', 'Squamous Cell Lung Carcinoma']
df_model_info_lung = df_model_info.where(df_model_info["cancer_type"].isin(lung_cancer)).dropna()

df_model_info_lung.head(5)

,model_id,tissue,cancer_type,tissue_status,cancer_type_detail
13,SIDM01387,Lung,Small Cell Lung Carcinoma,Metastasis,Small Cell Lung Carcinoma
18,SIDM00990,Lung,Non-Small Cell Lung Carcinoma,Metastasis,Non-Small Cell Lung Carcinoma
20,SIDM00019,Lung,Non-Small Cell Lung Carcinoma,Metastasis,Papillary Lung Adenocarcinoma
28,SIDM00046,Lung,Non-Small Cell Lung Carcinoma,Unknown,Lung Adenocarcinoma
30,SIDM01069,Lung,Non-Small Cell Lung Carcinoma,Tumour,Lung Adenocarcinoma


In [10]:
# List of model_ids with lung cancer
model_ids_lung = df_model_info_lung["model_id"].to_list()

## Expression data

cancerous RNA Seq Data from from different cell models. 

**columns:**
- id
- gene_symbol
- dataset_id
- data_source: Organisation or Project that provided the data
- model_id: gezüchtete Zelle / Zelllinie (ist die "Probe")
- tpm

# Data from Cell Modell Passport
Link: https://cellmodelpassports.sanger.ac.uk/downloads

Expression Data → all RNA Seq processed Data

All tissues are cancer tissues. We need to filter for lung cancer.

**Output file format:**
* id
* name
* tpm

In [11]:
# read in the data
file = "../import_data/CMP/rnaseq_all_data_20220624.csv"
df_cmp_all = pd.read_csv(file, delimiter=",", usecols=["gene_id", "gene_symbol", "model_id", "tpm"])

print("There are {} rows in the import_data.".format(df_cmp_all.shape[0]))

df_cmp_all.head()

There are 53348469 rows in the import_data.


,model_id,gene_id,tpm,gene_symbol
0,SIDM01313,SIDG03516,14.41,CASP10
1,SIDM01313,SIDG21420,0.64,NBPF10
2,SIDM01313,SIDG31799,0.39,RPL17P51
3,SIDM01313,SIDG25351,0.00,PPATP2
4,SIDM01313,SIDG19863,3.32,MMP28


### Clean Dataframe

In [12]:
df_cmp_all.rename(columns={"gene_symbol": "gene_name"}, inplace=True)

In [13]:
# filter rows with lung cancer model ids
df_cmp_all = df_cmp_all.where(df_cmp_all["model_id"].isin(model_ids_lung)).dropna()

print("There are {} rows with lung cancer data.".format(df_cmp_all.shape[0]))

There are 7564389 rows with lung cancer data.


### Analyze Dataset

In [15]:
# check for missing values
missing_values = df_cmp_all.isnull().sum()

# TPM Ranges
min_tpm = df_cmp_all["tpm"].min()
max_tpm = df_cmp_all["tpm"].max()

# genes
n_genes = df_cmp_all["gene_name"].nunique()

# tissues
n_tissues = df_cmp_all["model_id"].nunique()
df_cmp_all.drop(columns=["model_id"], inplace=True)

print(f"Missing values:\n"
      f"{missing_values}\n")

print(f"Min TPM: {min_tpm}")
print(f"Max TPM: {max_tpm}\n")

print(f"Number of genes: {n_genes}")
print(f"Number of tissues: {n_tissues}")

KeyError: 'model_id'


We then merged this file with our CMP data on the gene names to retrieve the ENS IDs for each gene.

After merging the data, we found that 3,760 genes had no ENS ID associated with them.
Since these genes were likely duplicates or did not exist in the Ensemble file, we removed them from our dataset to ensure consistency and accuracy of our analysis.
### Group Data to mean values

In [17]:
df_cmp_group = df_cmp_all.groupby(["gene_name", "gene_id"]).mean().reset_index()

print("There are {} rows in the grouped dataset.".format(df_cmp_group.shape[0]))
df_cmp_group

There are 37263 rows in the grouped dataset.


,gene_name,gene_id,tpm
0,A1BG,SIDG00001,0.827192
1,A1BG-AS1,SIDG00002,4.676305
2,A1CF,SIDG00003,1.355369
3,A2M,SIDG00004,1.669212
4,A2M-AS1,SIDG00005,1.033596
...,...,...,...
37258,ZYG11B,SIDG42471,21.124039
37259,ZYX,SIDG42472,99.636946
37260,ZYXP1,SIDG42473,0.000000
37261,ZZEF1,SIDG42474,17.697980


## Ensemble Dataset
Downloaded via Biomart



In [18]:
df_ensembl = pd.read_csv("../import_data/ENSEMBLE/ensemble_gene_id.txt", delimiter="\t")
df_ensembl.rename(columns={"gene_symbol": "gene_name"}, inplace=True)

# drop rows without gene_symbol
df_ensembl.drop(df_ensembl[df_ensembl["gene_name"].isnull()].index, inplace=True)

df_ensembl

,Gene_stable_ID,gene_name
0,ENSG00000210049,MT-TF
1,ENSG00000211459,MT-RNR1
2,ENSG00000210077,MT-TV
3,ENSG00000210082,MT-RNR2
4,ENSG00000209082,MT-TL1
...,...,...
70606,ENSG00000232679,LINC01705
70607,ENSG00000200033,RNU6-403P
70608,ENSG00000228437,LINC02474
70609,ENSG00000229463,LYST-AS1


In [19]:
duplicate_names = df_ensembl["gene_name"].duplicated(keep=False).sum()
rows = df_ensembl.shape[0]
print(f'{duplicate_names} from {rows} do not have a unique gene names in ENS Dataset')

10605 from 48311 do not have a unique gene names in ENS Dataset


PROBLEM: There are gene names that are not unique.

→ If the names are not unique, we cannot merge the data on the gene names with our dataset.

In [20]:
# delete all rows with not unique gene names
df_ensembl_unique = df_ensembl.drop_duplicates(subset=["gene_name"], keep=False)

df_ensembl_unique

,Gene_stable_ID,gene_name
0,ENSG00000210049,MT-TF
1,ENSG00000211459,MT-RNR1
2,ENSG00000210077,MT-TV
3,ENSG00000210082,MT-RNR2
4,ENSG00000209082,MT-TL1
...,...,...
70606,ENSG00000232679,LINC01705
70607,ENSG00000200033,RNU6-403P
70608,ENSG00000228437,LINC02474
70609,ENSG00000229463,LYST-AS1


## Merge Data

In [21]:
df_cmp_ens = pd.merge(df_cmp_group, df_ensembl_unique, on="gene_name", how="left")
df_cmp_ens

,gene_name,gene_id,tpm,Gene_stable_ID
0,A1BG,SIDG00001,0.827192,ENSG00000121410
1,A1BG-AS1,SIDG00002,4.676305,ENSG00000268895
2,A1CF,SIDG00003,1.355369,ENSG00000148584
3,A2M,SIDG00004,1.669212,ENSG00000175899
4,A2M-AS1,SIDG00005,1.033596,ENSG00000245105
...,...,...,...,...
37258,ZYG11B,SIDG42471,21.124039,ENSG00000162378
37259,ZYX,SIDG42472,99.636946,NaN
37260,ZYXP1,SIDG42473,0.000000,ENSG00000274572
37261,ZZEF1,SIDG42474,17.697980,ENSG00000074755


In [22]:
# check Data with missing ENS
missing_ens = df_cmp_ens[df_cmp_ens["Gene_stable_ID"].isnull()]

print(len(missing_ens), "/",len(df_cmp_ens),  "still have no ENS ID")
missing_ens

3760 / 37263 still have no ENS ID


,gene_name,gene_id,tpm,Gene_stable_ID
8,A2MP1,SIDG00009,0.035025,NaN
12,AAAS,SIDG00017,46.539015,NaN
14,AACSP1,SIDG00020,2.496207,NaN
16,AADACL2,SIDG00022,0.035813,NaN
17,AADACL2-AS1,SIDG00023,0.550640,NaN
...,...,...,...,...
37198,ZP2,SIDG42408,0.042167,NaN
37208,ZRANB2-AS2,SIDG42419,0.264926,NaN
37219,ZSCAN2,SIDG42425,6.817094,NaN
37239,ZSWIM4,SIDG42451,11.950049,NaN


TODO: Which of these 3760 genes do not have a ENS ID because of duplcate names and which did not occur from the beginning?

→ These genes with no ENS ID could get an ID by looking for ENS name aliases. But this makes tbe duplicate of names even worse.

Solution: Since these are only a few gene, we will drop these rows.

In [23]:
# show rows with duplicate names in df_cmp_ens
df_cmp_ens[df_cmp_ens["gene_name"].duplicated(keep=False)]

,gene_name,gene_id,tpm,Gene_stable_ID
30136,SEPTIN4,SIDG03156,0.065764,ENSG00000108387
30137,SEPTIN4,SIDG34083,1.701675,ENSG00000108387


### Clean up

In [24]:
df_cmp_ens.dropna(subset=["Gene_stable_ID"], inplace=True)
df_cmp_ens.rename(columns={"Gene_stable_ID": "Gene ID", 
                           "tpm":"cancerous TPM", 
                           "gene_name": "Gene Name" }, inplace=True)

# reorder columns
df_cmp = df_cmp_ens[['Gene ID', 'Gene Name', 'cancerous TPM']]

df_cmp

,Gene ID,Gene Name,cancerous TPM
0,ENSG00000121410,A1BG,0.827192
1,ENSG00000268895,A1BG-AS1,4.676305
2,ENSG00000148584,A1CF,1.355369
3,ENSG00000175899,A2M,1.669212
4,ENSG00000245105,A2M-AS1,1.033596
...,...,...,...
37257,ENSG00000232242,ZYG11AP1,0.000887
37258,ENSG00000162378,ZYG11B,21.124039
37260,ENSG00000274572,ZYXP1,0.000000
37261,ENSG00000074755,ZZEF1,17.697980


### Save Data

In [25]:
df_cmp.to_csv("../processed_data/CMP_cancer_mean.csv", index=False)
print(f'There are {df_cmp.shape[0]} rows/genes in the saved dataset.')

There are 33503 rows/genes in the saved dataset.


In [26]:
dfi.export(df_cmp.head(5), "../tex/figures/03_01_CMP_cancer_mean.png")